# Assignment 1

#### Student ID: *Double click here to fill the Student ID*

#### Name: *Double click here to fill the name*

In [ ]:
!pip install --pre pycaret[full] -qq
!pip install cleanlab -qq
!pip install modAL -qq
!pip install snorkel -qq

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


import matplotlib as mpl
from matplotlib import pyplot as plt
%matplotlib inline

## Q1: Low-code Machine Learning with Ames housing dataset

In this question, we are going to examine the low-code ML framework. Assuming we are assigned to deal with a different housing dataset after we finish the California census data project.

We will use a modified version of the Ames housing dataset. The original data was compiled by Dean De Cock for use in data science education and published in [De Cock, D. (2011)](https://www.tandfonline.com/doi/abs/10.1080/10691898.2011.11889627). The modified version contains 2930 rows with 79 columns describing every aspect of residential homes in Ames, Iowa.  We will only use a subset of features and be sure to set the random seed to 2023 in the following problems.

Firstly, execute the following code snippet for data preparation:

In [ ]:
import numpy as np
import pandas as pd

from pycaret.regression import *

# Prepare data
df = pd.read_csv("ames.csv")            
low_cardinality_cols = [cname for cname in df.columns if df[cname].nunique() < 10 and 
                        df[cname].dtype == "object"]
# Select numeric columns
numeric_cols = [cname for cname in df.columns if df[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
df = df[my_cols].copy()
df.head()

(a) Split 5% of data as the test set for testing purposes using `train_test_split()`. (5%)

In [ ]:
# coding your answer here.

(b) Use the `setup()` function to prepare the dataset for our ML model by fulfilling the following requirements: (10%)

* Features: All features in the `df` data frame, except the `SalePrice`
* Target: `SalePrice` 
* Split 25% of the data into the validation set
* Perform standardization for all features (You can assume that the data type belongs to `object` are categorical variables; otherwise, it is numerical variables)
* Perform one-hot encoding for all categorical features (`PyCaret` will discard one variable after one-hot encoding if the feature only has two levels and is regarded as an ordinal feature automatically, you do not have to change this behavior)

Hint: You may find the documentation (https://pycaret.readthedocs.io/en/latest/api/regression.html) useful. Note that PyCaret will [scale all features by default](https://github.com/pycaret/pycaret/issues/3076).

In [ ]:
# coding your answer here.

(c) Use `compare_models()` to perform 5-fold cross-validation by comparing the following models and selecting the best three models according to the MAE (mean absolute error) measure. (5%)

- Linear regression, Lasso regression, ridge regression, decision tree regressor, k-nearest neighbor regressor, random forest regressor, xgboost, lightgbm, and catboost.

List the top 3 best models according to the MAE measure.

In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*

(d) Choose the best model from (c) using `create_model()` and report the MAE on the test set in (a). (5%)

In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*

## Q2: Dealing with unlabeled dataset

In this question, we will practice the data preparation skill often used in a real-world project.

Assuming in the startup company, we are given an unlabeled dataset `data.csv` and a crowdsourcing labels `crowdsourcing.csv` from 50 workers. The data contains two features and 247 sample points. Each sample point belongs to one of 3 different classes. This is a practice project. Therefore we have access to the ground truth label `labels.csv`, **but the label can only be used to measure the accuracy, answer the queries from the active learning model, or initialize the active learning model. It can not be used in all the other training processes.** Be sure to set the random seed to 2023 in the following problems.

Firstly, read the dataset and divide it into training and testing sets using the following code snippet:

In [ ]:
X = pd.read_csv('data.csv', names=[0,1])
true_labels = np.loadtxt("labels.csv", delimiter=',')
multiannotator_labels = pd.read_csv('crowdsourcing.csv')

In [ ]:
X_train, X_test, multiannotator_labels, ano_unseen, Y_train, Y_test  = train_test_split(X, multiannotator_labels, true_labels, test_size=0.25, random_state=2023)

(a) We decide to use active learning first. Try randomly selecting one sample from each class in the training set and putting the remaining dataset into the query pool. Use the above three training samples and the corresponding training labels to train the following logistic regression classifier with L2 penalty: (10%)

```python
LogisticRegression(solver='liblinear', penalty='l2', C=10, random_state=2023)
```

Report the classification accuracy on the test set for this classifier. 

In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*

(b) Now use the model in (a) and run the active learning with pool-based sampling for 20 epochs using the `ActiveLearner` from [`modAL`](https://modal-python.readthedocs.io/en/latest/). The query strategy should be set to `uncertainty_sampling`, and you can use the ground truth label to answer the query. (10%)

Report the final classification accuracy on the test set and plot the line plot for the classification accuracy on the test set versus the epochs.

In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*

(c) In the second trial, we will use the `Snorkel` to explore the crowdsourcing label. The idea is that each crowd worker can be considered a single labeling function, as each worker labels a subset of data points and may have errors or conflicting labels with other workers. The procedure is as follows: (10%)

- Write the labeling function and apply the labeling function to the training set. You can also directly convert the crowdsourcing data frame to a matrix
- Train the `LabelModel` using the crowdsourcing matrix obtained above (Note that you should not use the ground truth label here) and generate the prediction for each training sample
- Finally, calculate the accuracy of the label on the training set using the ground truth training label

Note that you need to use the following hyperparameters when training the [`LabelModel`](https://snorkel.readthedocs.io/en/master/packages/_autosummary/labeling/snorkel.labeling.model.label_model.LabelModel.html):

```
fit(L_train, n_epochs=500, seed=2023, log_freq=20, l2=0.1, lr=0.001)
```

Hint: If you want to write the label function, you can create one labeling function per worker, return the label the worker submitted for a given sample, and abstain if they didn’t submit a label.

In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*

(d) In the third trial, we will use the [`cleanlab`](https://github.com/cleanlab/cleanlab) to explore the crowdsourcing label. The procedure is as follows: (10%)

- Firstly, get the majority vote labels using `get_majority_vote_label()` and compute out-of-sample predicted probabilities using 5-fold cross-validation for each sample using the training set with the majority vote labels. The model you use here should be the same as the one you use in (a)
- Use the `get_label_quality_multiannotator()` from cleanlab to get consensus labels by combining the out-of-sample predicted probabilities and the crowdsourcing label
- Finally, calculate the accuracy of both the majority and consensus label on the training set using the ground truth training label

In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*

(e) Train three logistic regression classifiers with the same hyperparameters as (a) using the labels obtained from `Snorkel` in (c) and the majority-vote and consensus labels from (d). Report the classification accuracy on the test set. Which one performs best? (5%)

In [ ]:
# coding your answer here.

> Ans: *double click here to answer the question.*

## Q3: Analyze Github repositories dataset using SQL

Kaggle has a rich number of [BigQuery](https://www.kaggle.com/datasets?fileType=bigQuery) and [SQLite](https://www.kaggle.com/datasets?fileType=sqlite) datasets that you can practice your SQL skill.

Assuming in the startup company, we join another project which aims to analyze a big dataset. The dataset is related to Github.  GitHub is the most popular place to collaborate on software projects. A GitHub **repository** (or **repo**) is a collection of files associated with a specific project. Most repos on GitHub are shared under a specific legal license, which determines the legal restrictions on their use. In addition, each project will be written using several programming languages. Finally, there are several statistics that show the popularity of repositories, like the watch number.

<p align="center">
<img src="https://drive.google.com/uc?id=1Z7ubmkY9JQMdHxAHD5O33gYJ0tNS9O-O" alt="drawing" width="1000"/>
</p>

In this example, we are asked to analyze the GitHub repositories dataset provided by our customer and it is hosted on https://www.kaggle.com/datasets/github/github-repos?select=languages.

Hint: Refer to our laboratory for useful commands.

Firstly, if you are using Colab, use the following code snippet to set up the client. For more detail, please refer to our lab.

In [ ]:
# @title Setup
from google.cloud import bigquery
from google.oauth2 import service_account

# TODO(developer): Set key_path to the path to the service account key file.

key_path = "lunar-pact-378812-7a28b789bde2.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

(a) In the first stage, explore the dataset using the following procedure: (10%)

- Print the names of all tables available in the dataset. 
- Print the table schema of the `languages` , `sample_repos` and the `licenses` tables.
- Preview the first ten rows of the above three tables by retrieving the data frame

Hint: the dataset can be access via `dataset_ref = client.dataset("github_repos", project="bigquery-public-data")`

In [ ]:
# coding your answer here.

(b) The `licenses` table provides information about the license used by each repository. Please write a query that returns a data frame that contains two columns by following the restrictions below: (10%)

* The first column is `license` from the `licenses` table (Remember the `SELECT... from` clause)
* The second column is `number_of_repos`, which is the number of repositories belonging to each license. (Remember the `COUNT` and `GROUP BY` clauses)
* Filter out the rows whose `number_of_repos` is smaller or equal to 50,000 (Remember the `HAVING` clause)
* Sort the resulting data frame by the `number_of_repos` column in descending order (Remember the `ORDER BY` clause)


In the retrieved data frame, which license is most popular?

In [ ]:
# coding your answer here.
query = """
        """

############   Do not modify the code below     ############ 
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
results = query_job.to_dataframe()
############   Do not modify the code above     ############ 

> Ans: *double click here to answer the question.*

(c) The `sample_repos` table contains information about the popularity of each repository. In addition, the `languages` table includes information on the programming language used by each repository. (10%)

Now, write a query that has a single row for each repository. Your results should have three columns (Remember the `SELECT...WHERE` clause) and follow the below restrictions: 

* Restrict the results to rows containing the word 'Python' in the `language` column of the `languages` table. (Remember the `SELECT...WHERE` clause and remember to use `UNNEST` clause)
* The first column `repo_name` contains the `repo_name` column from the `sample_repos` table (Remember the `INNER JOIN` clause)
* The second column `watch_count` contains the `watch_count` column from the `sample_repos` table 
* The third column `language` contains the `language` column from the `languages` table 
* Sort the resulting data frame by the `watch_count` column in descending order (Remember the `ORDER BY` clause)

In the retrieve data frame, which is the most popular repository that is written (Partially or totally) by Python?

Hint: Notice that in the table schema of `language` table, `language` is nested data with the type `REPEATED`.

In [ ]:
# coding your answer here.
query = """
        """

############   Do not modify the code below     ############ 
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
results = query_job.to_dataframe()
############   Do not modify the code above     ############ 

> Ans: *double click here to answer the question.*